In [288]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
# mediapipe to extract keypoints and values from the hand regions
import mediapipe as mp

In [289]:
# pip install opencv-python

In [290]:
# pip install tensorflow

In [291]:
# pip install scikit-learn

In [292]:
#pip install mediapipe

## Keypoints using Mediapipe holistic 

In [293]:
# mediapipe holistic for downloading and leveraging the model
mpHolistic = mp.solutions.holistic

# mediapipe drawing utilities for drawing the keypoints on the face and hands
mpDrawing = mp.solutions.drawing_utils

In [294]:
# read the image and make predictions
def mpDetection(image, model):
    
    # convert BGR image format into RGB as open-cv reads the image data in RGB, however, mediapipe detects image in RGB format
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # make the image unwritable for saving the memory consumption
    image.flags.writeable = False
    
    # make the prediction using mediapipe
    results = model.process(image)
    
    # make the image writable
    image.flags.writeable = True
    
    # again, convert the image format into original ==> RGB to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    return image, results

In [295]:
# draw the landmarks on the face and hands

def drawLandmarks(image, results):
    # mpDrawing.draw_landmarks(image, results.face_landmarks, mpHolistic.FACE_CONNECTIONS)
    # draw the landmarks by passing image, landmarks and connections in the draw_landmarks function
    mpDrawing.draw_landmarks(image, results.pose_landmarks, mpHolistic.POSE_CONNECTIONS)
    mpDrawing.draw_landmarks(image, results.left_hand_landmarks, mpHolistic.HAND_CONNECTIONS)
    mpDrawing.draw_landmarks(image, results.right_hand_landmarks, mpHolistic.HAND_CONNECTIONS)


In [296]:
# add colors on the landmarks
def drawColorfulLandmarks(image, results):
    
    
    # add grey color on the pose (between wrist and shoulder)
    mpDrawing.draw_landmarks(image, results.pose_landmarks, mpHolistic.POSE_CONNECTIONS, 
                               mpDrawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mpDrawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                            )
    # add pink color on the landmarks of the left hand
    mpDrawing.draw_landmarks(image, results.left_hand_landmarks, mpHolistic.HAND_CONNECTIONS, 
                              mpDrawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mpDrawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                            )
    
    # add blue color on the landmarks of the right hand
    mpDrawing.draw_landmarks(image, results.right_hand_landmarks, mpHolistic.HAND_CONNECTIONS, 
                               mpDrawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mpDrawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                            )
    

In [ ]:
# capture camera
cap = cv2.VideoCapture(0) 
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# set the mediapipe model
with mpHolistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # loop the frames of the webcam
    while cap.isOpened():
        # read the frame/image from the webcam
        ret, frame = cap.read()

        # make the detection
        image, results = mpDetection(frame, holistic)
        # print(results)
        
        # draw the landmarks
        drawColorfulLandmarks(image, results)


        # check if the frame was read successfully
        if not ret:
            print("Error: Could not read frame.")
            break

        # show the window
        cv2.imshow('Hand Gesture Recognition System with LSTM', image)

        # break the loop when 'q' key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
                break

# release the webcam
cap.release()

# close all OpenCV windows
cv2.destroyAllWindows()

In [ ]:
# visualizing the last image/frame
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))


In [ ]:
# drawing the landmarks on the frame without styling
drawLandmarks(frame, results)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [ ]:
# drawing the landmarks on the frame with styling
drawColorfulLandmarks(frame, results)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [ ]:

print('Face landmarks number: ', len(results.face_landmarks.landmark) )  # 468

print('Pose landmarks number: ', len(results.pose_landmarks.landmark) )  # 33

print('Right landmarks number: ', len(results.right_hand_landmarks.landmark) )  # 21

# print('Left landmarks number: ', len(results.left_hand_landmarks.landmark) )  # 21



In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
results.face_landmarks.landmark

In [ ]:
mpHolistic.HAND_CONNECTIONS